# CSCI 381/780 (Fall 2022) - Project 2

**Due Date: Monday, November 14 by 4 PM**

## Description
In this project you will construct machine learning models on two different real-world datasets using unsupervised learning and regression.

## Instructions
1. In this project you will write code to construct machine learning models and write responses to questions concerning the performance of said models. Please complete all sections below, adding new *Code* or *Markdown* cells as appropriate to answer the questions.
2. There are many Scikit-learn functions that leverage randomness to generate results. For these functions, a pseudorandom generator can be initialized using a seed value by passing the parameter `random_state=XXX`, where `XXX` is some number between 1 and 2^31-1. For each of these functions, **you will utilize your CUNY ID number** to initialize the function. Functions include:
- `ShuffleSplit`
- `KFold`
- `KMeans`
- `GridSearchCV`
- `Lasso`
- `MLPRegressor`
3. You will **work independently** on the project. Please make use of the *Python Data Science Reference Materials* posted on Blackboard, or **come to office hours should you need further assistance**.
4. You will submit a single Jupyter notebook containing all code and written responses via Blackboard by the due date listed above.

## Grading

### Running Code
Your Jupyter notebook must be able to run from start to finish **without error**. Please turn any cell that contains scratch work or other non-executable items to *Raw*. **Notebooks that cannot run to completion will receive a grade of 0**.


### Rubric

|**Part**|1.1|1.2|2.1|2.2|2.3|2.4|2.5|2.6|**Total**|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|**%**|10|30|15|5|10|10|10|10|100|


# Part 1: Clustering

In this part of the project, you will be using data collected by the US Geological Survey and the US Forest Service. The data describes various cartographic and geologic features related to forest cover in US wilderness areas, with each sample representing a 30 x 30 meter cell.

The **goal** will be to *identify clusters* that represent forest cover types, and using these clusters *extract relationships* between forest covers and the provided cartographic/geologic features.

In [16]:
#library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [17]:
# load the dataset
cover_column_names=["Elevation","Aspect","Slope",
              "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",
              "Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon",
              "Hillshade_3pm","Horizontal_Distance_To_Fire_Points"]
wilderness_cols =["Wilderness_"+str(i) for i in range(4)]
soil_col =["Soil_"+str(i) for i in range(40)]
cover_columns_names = cover_column_names + wilderness_cols + soil_col

## 1.1 Load Data
Set the variable `COVER_FILE` to the **full path** to the forest cover dataset (**forest_cover_dataset.csv**) on your system. Load the file into a dataframe (you may initialize the column names using the header list `cover_column_names`), then:
1. Determine the number and types of features.
2. Perform a **ShuffleSplit** of the data into training/validation/test sets, 60%/20%/20%.
3. **Center** the training/validation/test splits (fit on the training set, then transform the validation/test sets).

In [18]:
# cover_file = f'C:/Users/tanzi/CS Lang IDE/PycharmProjects/Project-2/UTF-8_forest_cover_dataset.csv'
#search later, how to auto complete full path. https://www.youtube.com/watch?v=3Spa10-mwsw

cover_file = f'C:/Users/tanzi/CS Lang IDE/PycharmProjects/Jonathan/Project2/Project-2/UTF-8_forest_cover_dataset.csv'
forest_data = pd.read_csv(cover_file, names=cover_columns_names)
forest_data


,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_30,Soil_31,Soil_32,Soil_33,Soil_34,Soil_35,Soil_36,Soil_37,Soil_38,Soil_39
0,2825,265,17,319,108,2298,174,245,209,342,...,0,0,1,0,0,0,0,0,0,0
1,2719,198,34,134,76,2352,187,245,167,693,...,0,0,0,0,0,0,0,0,0,0
2,3146,152,14,212,41,940,237,239,130,2007,...,0,0,0,0,0,0,0,0,0,0
3,2882,18,18,95,-3,485,203,199,128,342,...,1,0,0,0,0,0,0,0,0,0
4,2912,349,17,283,57,2729,187,210,160,2201,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280246,2919,224,18,85,24,451,193,254,193,1436,...,0,0,0,0,0,0,0,0,0,0
280247,2151,114,29,42,29,499,254,197,49,731,...,0,0,0,0,0,0,0,0,0,0
280248,3008,221,14,418,99,5346,202,253,185,1300,...,0,0,0,0,0,0,0,0,0,0
280249,2923,58,12,134,12,2213,228,214,117,424,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Determine the number of features
num_features = len(cover_columns_names)
print('Number of features: ', num_features)

Number of features:  54


In [20]:
#change the variable name

#Determine nature of object types
crime_cat_columns=dict()
crime_cat_columns_idx=dict()
crime_num_columns_idx=dict()
for col in cover_columns_names:
    if forest_data[col].dtype == 'int64':
        crime_cat_columns[col]=forest_data[col].unique()
        crime_cat_columns_idx[col]=forest_data.columns.get_loc(col)
        print(col,":", crime_cat_columns[col])
    elif np.issubdtype(forest_data.at[0,col], np.number):
        crime_num_columns_idx[col]=forest_data.columns.get_loc(col)

Elevation : [2825 2719 3146 ... 3626 3541 3607]
Aspect : [265 198 152  18 349 202 102  90   9 117 315  40 110   3 359 106 108 150
  32 347 127  62 123 176  53 100 180 166 189 243 294  72 112  22  45 274
  60 344  28 299  65  34 169  73  43  82 262 232  83  37 329  49 269  66
 140 133  33  61 350  63  74 326 193 225 226 342 318 276 316 167 303  55
 131  69 251  19 220  52 135  11 270  26  14 357  59  42  51 158 204 138
 356  78  12 306 165 238 235 352 219 307 119 311 351  36  95  25 163 162
 272  17 340 113  41 348   0 324  35  68  13  20 334 343 304  85  96 146
  99 143 301  79  98 336 196  21 101 116 170 317 245  38  92 111 211 354
 121   8 249 157 277  31   4 181 322 105 288 332 185 187 241 199 172 289
 207 281 291 222 107  93 302 195  56 247 314 327   7 287 263 333 309 209
 148 331   1 266   5  91  15 268  16 145 184 254 194  71 337 183 346 149
  80 285  58  57  89 118 330  50 125 296 182  27 231 248  54  70  97 103
 156 147  76  23  67 338 256 174 323 358 177 267 300 215  30 206 20

In [21]:
# Perform a **ShuffleSplit** of the data into training/validation/test sets, 60%/20%/20%.
forest_data_list = np.array(forest_data)
from sklearn.model_selection import ShuffleSplit
split_test = ShuffleSplit(n_splits=1,test_size=0.2, random_state=23508893)
for train_index, test_index in split_test.split(forest_data_list):
    forest_train_proj_full = forest_data_list[train_index,:]
    forest_test_proj = forest_data_list[test_index,:]


split_validation = ShuffleSplit(n_splits=1,test_size=0.25, random_state=23508893)
for train_index, validation_index in split_validation.split(forest_train_proj_full):
    forest_train_proj = forest_train_proj_full[train_index,:]
    forest_validation_proj = forest_train_proj_full[validation_index,:]


In [ ]:
# #Maybe comment this out?
# print("Dataset\t\tPercent")
# print("Training\t" + str(np.sum(forest_train_proj) / len(forest_train_proj)))
# print("Validation\t" + str(np.sum(forest_validation_proj) / len(forest_validation_proj)))
# print("Test\t\t" + str(np.sum(forest_test_proj) / len(forest_test_proj)))

1. How would want us to solve 1.1.3 [center the training]. Should I use customs transformer? If so, during the next step, should I use data_centered_train or forest_train_proj?
2. forest_train_proj = dataset (X) | data_center_train = first center randomly
3.


In [22]:
# Customs transformers
from sklearn.preprocessing import FunctionTransformer
center_function = lambda x: x-x.mean()
transformer = FunctionTransformer(center_function, validate=True)

In [23]:
# **Center** the training/validation/test splits (fit on the training set, then transform the validation/test sets).
# use this data for clustering. Ask the professor if I should use data_centered or not_center
# Copy the standard scaler from part 1
data_centered_train = transformer.fit_transform(forest_train_proj)
data_centered_valid = transformer.transform(forest_validation_proj)
data_centered_test = transformer.transform(forest_test_proj)

data_centered_full_train = transformer.transform(forest_train_proj_full)

data_centered_test

array([[2868.87819823,  160.87819823, -151.12180177, ..., -154.12180177,
        -154.12180177, -154.12180177],
       [2777.87819823,   48.87819823, -140.12180177, ..., -154.12180177,
        -154.12180177, -154.12180177],
       [2378.87819823,   60.87819823, -125.12180177, ..., -154.12180177,
        -154.12180177, -154.12180177],
       ...,
       [2783.87819823,  125.87819823, -130.12180177, ..., -154.12180177,
        -154.12180177, -154.12180177],
       [2359.87819823,  152.87819823, -132.12180177, ..., -154.12180177,
        -154.12180177, -154.12180177],
       [3028.87819823,   38.87819823, -146.12180177, ..., -154.12180177,
        -154.12180177, -154.12180177]])

In [24]:
# # sns.pairplot(forest_train_proj[])
# plt.plot(forest_train_proj)
# plt.show()

## 1.2 $k$-Means Clustering

1. Cluster the *training* dataset using ```KMeans``` from Scikit-learn using the values for $k$ below. You can utilize the default implementation in Scikit-learn, which is $k$-means$++$. Construct models for each value of $k$ specified in the ``k_values`` array below.
2. Apply the trained models to the validation dataset.
3. Compute the **mean** inertia for each value of $k$ on the training and validation datasets. For the training data, you can extract the *total* (**unnormalized**) inertia from the trained models via their respective ```inertia_```. For the validation models, you can utilize the function ```total_inertia``` provided below.
4. Plot the mean inertias for all values of $k$ and for both the training and validation sets on a single plot. Based on the elbow method, which value of $k$ should be chosen so that the model will generalize to new data?
5. The *Silhouette Coefficient* is another cluster performance metric that combines intra-cluster distance with inter-cluster distances from clusters in close proximity to each other. Utilize Scikit-learn's ```silhouette_score``` function [(documentation)](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html#sklearn.metrics.silhouette_score) to compute the silhouette coefficient for all values of $k$ on the *validation* set. Plot these values and determine via the elbow method which value of $k$ should be chosen. How does this value compare to the one chosen using mean inertia?
6. Using whichever value of $k$ you think is the best based on steps 4) and 5), create a plot for each feature that contains [boxplots](https://matplotlib.org/stable/gallery/statistics/boxplot_demo.html#sphx-glr-gallery-statistics-boxplot-demo-py) of that feature for each cluster. Based upon these plots, determine which features you think are meaningful in discriminating between the clusters, as well as any other trends you observe.

### Please Read!
You may benefit from parallelizing the creation of the $k$-means models by using the Python package ```joblib```. I've included a code template below that you can customize. For additional details, please refer to [joblib's documentation](https://joblib.readthedocs.io/en/latest/parallel.html).

## Cluster the *training* dataset using ```KMeans```. Construct models for each value of $k$ specified in the ``k_values`` array
### Comment:
1. Should I use fit or fit_transform?
2. fit_predict() vs predict() <- I think I got my answer| labels vs predict; what is labels?
3. whats the difference between writing out the parameter for iniit: 'kmean++' or not?
The reason why I ask, because I don't see any data difference with or without, even after adding n_init=k, where is interger of the list from k_value

In [25]:
k_values=list(range(2,11))+[15,20]

#n-clusters = int [not list]; figure out a way to get int.
from sklearn.cluster import KMeans
#fit: kmeans return type: kmeans

for k in k_values:
    kmeans = KMeans(n_clusters=k,random_state=23508893).fit(forest_train_proj) # fit vs fit_transform?


In [26]:
kmeans.cluster_centers_

array([[ 3.13485042e+03,  1.80206424e+02,  1.11966619e+01, ...,
         7.46339159e-02,  1.05495198e-02,  5.89805982e-17],
       [ 2.90396929e+03,  1.56102918e+02,  1.49163384e+01, ...,
         1.57549234e-02,  1.18161926e-02,  7.51276441e-03],
       [ 2.95963745e+03,  1.74061255e+02,  1.05181773e+01, ...,
         1.04083409e-17, -5.37764278e-17,  1.73472348e-17],
       ...,
       [ 2.98986947e+03,  1.48964153e+02,  1.37919242e+01, ...,
         1.78821590e-02,  4.36753193e-03,  3.90605686e-02],
       [ 2.95087006e+03,  1.81988272e+02,  1.42078879e+01, ...,
         1.66061235e-03,  1.97197717e-03,  9.34094447e-04],
       [ 3.05361060e+03,  1.14238266e+02,  1.09398159e+01, ...,
         2.62744217e-02,  3.32360207e-02,  1.54951718e-02]])

## Apply the trained models to the validation data

In [ ]:
# kmeans.fit_predict(forest_validation_proj)
# valid_predict = kmeans.predict(forest_validation_proj, train_predict)
valid_predict = kmeans.predict(forest_validation_proj)
valid_predict

In [ ]:
kmeans.cluster_centers_

### Comment
1. Is the func supposed to be for loop of the kvalues? and the data is train_model?
2. Create a function for for-loop, where it takes list, data

In [ ]:
# from joblib import Parallel, delayed, parallel_backend
#
# with parallel_backend("loky", inner_max_num_threads=2):
#     results = Parallel(n_jobs=4)(delayed(func)(x, y) for x, y in data)
#     # which data?
#     # || x = forest_train_proj and y = data_centered_train?
#     # || x = k-mean and y = k-mean ++?
#     # || x = forest_train_proj and y = k-means
#     # func = center_function? or [cluster_centers_ or labels_] from k-means?

In [ ]:
def total_inertia(dataset,centers,labels,distance=None,mean=True):
    # dataset= forest_validation_proj | centers = kmeans.cluster_centers_ [valid_predict] | label = valid_predict |mean=false
    """
    Computes the total inertia (the intracluster variance) given a dataset and the cluster centers.
    
    Parameters
    ----------
    dataset : numpy.ndarray
        The dataset to evaluate.
    center : numpy.ndarray or list
        The list of cluster centers.
    distance : function
        A function that computes the pairwise distance between two samples.
        Default: Euclidean (l_2) distance
    mean : bool
        If True, returns the mean inertia.
        Default: True
        
    Returns
    -------
    total_inertia : float
       The mean inertia (if mean==True) or total inertia (if mean==False).
"""
    num_samples,_ = dataset.shape
    total_inertia=0
    if distance is None:
        distance=lambda x,y: np.linalg.norm(x-y)**2
    for i,sample in enumerate(dataset):
        center = centers[labels[i]]
        total_inertia += distance(sample,center)
    
    if mean:
        return total_inertia/num_samples
    else:
        return total_inertia

## Compute the **mean** inertia for each value of k on the training and validation datasets.
1. For the training data, you can extract the *total* (**unnormalized**) inertia from the trained models via their respective ```inertia_```.
2. For the validation models, you can utilize the function ```total_inertia``` provided below.

### Comment:
1. Given ```total-inertia``` has way to computer mean intertia, if we set mean to true. If that's the case, is that what you want us to use for training data as well?
2. what do you mean by ***via their respective*** ```inertia_``` ?
3. Mean inertia for train model, work, I think?
4. But for the output for valid, the mean sse is coming the same


In [ ]:
kmeans.labels_

In [ ]:
# dataset= forest_train_proj | centers = kmeans.cluster_centers_ | labels = kmeans.labels | mean=true
# total_inertia(forest_train_proj,kmeans.cluster_centers_,kmeans.labels_,mean=True)

mean_train = []
for k in k_values:
    km= KMeans(n_clusters=k, random_state=23508893).fit(forest_train_proj)
    u = km.inertia_
    mean_train.append(u)
    print("The inertia for: ", k, "Clusters is:", u)

In [ ]:
#combine mean_valid and mean_train cell
# dataset= forest_validation_proj | centers = kmeans.cluster_centers_ | mean=false
# total_inertia(dataset,centers,labels,distance=None,mean=True)
# mean_valid = []
# for k in k_values:
#     forest_valid_inertia =total_inertia(forest_validation_proj,kmeans.cluster_centers_, kmeans.labels_, mean=True, distance=None)
#     km= KMeans(n_clusters=k).fit(forest_validation_proj)
#     mean_valid.append(forest_valid_inertia)

total_inertia(forest_validation_proj,kmeans.cluster_centers_, kmeans.labels_, mean=False, distance=None)

In [ ]:
plt.xlabel('K')
plt.ylabel('Sum of squared error')
plt.plot(k_values, mean_train, color = 'black', label ='mean_train')
# plt.plot(k_values, forest_valid_inertia, color = 'blue', label ='mean_valid')
plt.legend(['mean_train', 'mean_valid'])

## Plot the mean inertias for all values of $k$ and for both the training and validation sets on a single plot.
1. Based on the elbow method, which value of $k$ should be chosen so that the model will generalize to new data?

# Part 2: Regression

In this part of the project, you will be utilizing a US database of crime and law enforcement statistics broken down by US Census communities. The **goal** will be to create *regression models* that predict *per capita violent crimes* (the response variable `ViolentCrimesPerPop`) for a given community based on these inputs. 

In [3]:
crime_column_names= ['state', 'county', 'community', 'communityname', 'fold', 'population', 'householdsize',
                 'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'agePct12t21', 'agePct12t29',
                 'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome', 'pctWWage', 'pctWFarmSelf',
                 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst', 'pctWRetire', 'medFamInc', 'perCapInc', 'whitePerCap',
                 'blackPerCap', 'indianPerCap', 'AsianPerCap', 'OtherPerCap', 'HispPerCap', 'NumUnderPov', 'PctPopUnderPov',
                 'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed', 'PctEmploy', 'PctEmplManu',
                 'PctEmplProfServ', 'PctOccupManu', 'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr',
                 'FemalePctDiv', 'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par', 'PctYoungKids2Par',
                 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom', 'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent',
                 'PctImmigRec5', 'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5', 'PctRecImmig8',
                 'PctRecImmig10', 'PctSpeakEnglOnly', 'PctNotSpeakEnglWell', 'PctLargHouseFam', 'PctLargHouseOccup',
                 'PersPerOccupHous', 'PersPerOwnOccHous', 'PersPerRentOccHous', 'PctPersOwnOccup', 'PctPersDenseHous',
                 'PctHousLess3BR', 'MedNumBR', 'HousVacant', 'PctHousOccup', 'PctHousOwnOcc', 'PctVacantBoarded',
                 'PctVacMore6Mos', 'MedYrHousBuilt', 'PctHousNoPhone', 'PctWOFullPlumb', 'OwnOccLowQuart', 'OwnOccMedVal',
                 'OwnOccHiQuart', 'RentLowQ', 'RentMedian', 'RentHighQ', 'MedRent', 'MedRentPctHousInc', 'MedOwnCostPctInc',
                 'MedOwnCostPctIncNoMtg', 'NumInShelters', 'NumStreet', 'PctForeignBorn', 'PctBornSameState', 'PctSameHouse85',
                 'PctSameCity85', 'PctSameState85', 'LemasSwornFT', 'LemasSwFTPerPop', 'LemasSwFTFieldOps',
                 'LemasSwFTFieldPerPop', 'LemasTotalReq', 'LemasTotReqPerPop', 'PolicReqPerOffic',
                 'PolicPerPop', 'RacialMatchCommPol', 'PctPolicWhite', 'PctPolicBlack', 'PctPolicHisp',
                 'PctPolicAsian', 'PctPolicMinor', 'OfficAssgnDrugUnits', 'NumKindsDrugsSeiz', 'PolicAveOTWorked',
                 'LandArea', 'PopDens', 'PctUsePubTrans', 'PolicCars', 'PolicOperBudg', 'LemasPctPolicOnPatr',
                 'LemasGangUnitDeploy', 'LemasPctOfficDrugUn', 'PolicBudgPerPop', 'ViolentCrimesPerPop']

## 2.1 Load Data
Set the variable `CRIME_FILE` to the **full path** to the forest cover dataset (**crime.csv**) on your system. Load the file into a dataframe (you may initialize the column names using the header list `crime_column_names`), then:
1. Determine the number and types of features.
2. Perform a **ShuffleSplit** of the data into training/validation/test sets, 60%/20%/20%. 
3. Split the **non-test data** (*training* + *validation* data) into **5 folds** for cross-validation purposes.
4. Perform any necessary preprocessing on dataset. This may include:
  * determining if any features should be dropped;
  * handling missing data, through imputation and/or complete case analysis. If you **perform imputation on numerical values**, please use **median** imputation.

**Please note!** The Scikit-learn function `SimpleImputer` does not work as expected when the feature to impute is numerical but the missing values are not. One way to solve this is to first replace the missing values with NaN values (e.g., `np.nan`) using the Pandas Dataframe method `replace` [(documentation)](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html).

In [6]:
# crime_file = f'C:/Users/tanzi/CS Lang IDE/PycharmProjects/Project-2/UTF-8_crime.csv'
crime_file = f'C:/Users/tanzi/CS Lang IDE/PycharmProjects/Jonathan/Project2/Project-2/UTF-8_crime.csv'
crime_data = pd.read_csv(crime_file, names=crime_column_names)
crime_data

,state,county,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,?,?,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,...,0.01,0.28,0.05,?,?,?,?,0.00,?,0.09
1990,6,?,?,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,...,0.02,0.37,0.20,?,?,?,?,0.00,?,0.45
1991,9,9,80070,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,...,0.08,0.32,0.18,0.08,0.06,0.78,0,0.91,0.28,0.23
1992,25,17,72600,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,...,0.03,0.38,0.33,0.02,0.02,0.79,0,0.22,0.18,0.19


In [7]:
crime_data.dtypes

state                    int64
county                  object
community               object
communityname           object
fold                     int64
                        ...   
LemasPctPolicOnPatr     object
LemasGangUnitDeploy     object
LemasPctOfficDrugUn    float64
PolicBudgPerPop         object
ViolentCrimesPerPop    float64
Length: 128, dtype: object

In [8]:
# Determine the number of features
num_features = len(crime_column_names)
print('Number of features: ', num_features)

Number of features:  128


In [9]:
#Determine nature of object types
crime_cat_columns=dict()
crime_cat_columns_idx=dict()
crime_num_columns_idx=dict()
for col in crime_column_names:
    if crime_data[col].dtype == 'object':
        crime_cat_columns[col]=crime_data[col].unique()
        crime_cat_columns_idx[col]=crime_data.columns.get_loc(col)
        print(col,":",crime_cat_columns[col])
    elif np.issubdtype(crime_data.at[0,col], np.number):
        crime_num_columns_idx[col]=crime_data.columns.get_loc(col)

county : ['?' '5' '95' '7' '21' '87' '187' '1' '27' '129' '31' '193' '29' '680'
 '23' '3' '19' '35' '790' '53' '9' '119' '99' '17' '11' '135' '133' '45'
 '39' '61' '43' '157' '15' '13' '37' '107' '89' '41' '93' '71' '590' '55'
 '101' '75' '57' '63' '760' '69' '25' '830' '77' '155' '151' '123' '113'
 '49' '139' '169' '630' '79' '510' '65' '81' '105' '141' '33' '59' '131'
 '173' '109' '153' '103' '820' '750' '770' '740' '163' '97' '85' '121'
 '91' '73' '215' '181' '125' '690' '650' '127' '143' '660' '735' '147'
 '670' '683' '111' '67' '550' '117' '570' '800' '810' '700' '165' '167'
 '710' '145' '730' '840' '775']
community : ['?' '81440' '6096' '41500' '44105' '30075' '91370' '1000' '17650' '66376'
 '93926' '61798' '22490' '47672' '59280' '40560' '22960' '55950' '81740'
 '12900' '57660' '45120' '43620' '30570' '75216' '42750' '58200' '54837'
 '34950' '46520' '82525' '84000' '63968' '66700' '22240' '37875' '87560'
 '45140' '76490' '73060' '83080' '83432' '20100' '80240' '47616' '66060'
 '

# Preprocessing Dataset

## Removal of redundact data
# Comment:
1. 1st approach: debating between state and county vs commmunityname. I highly think community name should be removed, since it's a string.
but the reason why I did not remove it because as the description talked about *given community*, which is the input. or is community enough?
2. 2nd apporach:I am assuming community mean number of people are living in communityname. And that's why I was thinking of encoding communityname.
3. Question 1 and 2 got my answer.

In [10]:
#let's drop county as it is redundant
crime_data.drop('county', axis=1, inplace=True)
crime_column_names.remove('county')
#need to remap indices
crime_cat_columns = dict()
crime_cat_columns_idx = dict()
crime_num_columns_idx = []
for col in crime_column_names:
    if crime_data[col].dtype == 'object':
        crime_cat_columns[col] = crime_data[col].unique()
        crime_cat_columns_idx[col] = crime_data.columns.get_loc(col)
        print(col, ":", crime_cat_columns[col])
    elif np.issubdtype(crime_data.at[0, col], np.number):
        crime_num_columns_idx.append(crime_data.columns.get_loc(col))

community : ['?' '81440' '6096' '41500' '44105' '30075' '91370' '1000' '17650' '66376'
 '93926' '61798' '22490' '47672' '59280' '40560' '22960' '55950' '81740'
 '12900' '57660' '45120' '43620' '30570' '75216' '42750' '58200' '54837'
 '34950' '46520' '82525' '84000' '63968' '66700' '22240' '37875' '87560'
 '45140' '76490' '73060' '83080' '83432' '20100' '80240' '47616' '66060'
 '21504' '39300' '58730' '45056' '22456' '4825' '13690' '51660' '79740'
 '66660' '75740' '37000' '62432' '20080' '56550' '7310' '7665' '16495'
 '70020' '54040' '74880' '37490' '82000' '32296' '36000' '70380' '5320'
 '44700' '53104' '56966' '18070' '38715' '48955' '14875' '40620' '23832'
 '40115' '65280' '21344' '71990' '83975' '56979' '41216' '1798' '26275'
 '54640' '44328' '52070' '17475' '11950' '76030' '1465' '42168' '86772'
 '40775' '68460' '32328' '67000' '14160' '18640' '27025' '67945' '18455'
 '74190' '59980' '7200' '16775' '20352' '10750' '41310' '4500' '77570'
 '41610' '86160' '78736' '2000' '43344' '6579

## Handling missing data

In [11]:
#In order to handle missing data, Replace ? with Nan
crime_data.replace('?', np.nan, inplace= True)
crime_data

,state,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,NaN,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,NaN,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,...,0.02,0.12,0.45,NaN,NaN,NaN,NaN,0.00,NaN,0.67
2,24,NaN,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,...,0.01,0.21,0.02,NaN,NaN,NaN,NaN,0.00,NaN,0.43
3,34,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,...,0.02,0.39,0.28,NaN,NaN,NaN,NaN,0.00,NaN,0.12
4,42,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,...,0.04,0.09,0.02,NaN,NaN,NaN,NaN,0.00,NaN,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,NaN,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,0.16,...,0.01,0.28,0.05,NaN,NaN,NaN,NaN,0.00,NaN,0.09
1990,6,NaN,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,0.32,...,0.02,0.37,0.20,NaN,NaN,NaN,NaN,0.00,NaN,0.45
1991,9,80070,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,0.25,...,0.08,0.32,0.18,0.08,0.06,0.78,0,0.91,0.28,0.23
1992,25,72600,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,0.10,...,0.03,0.38,0.33,0.02,0.02,0.79,0,0.22,0.18,0.19


In [12]:
crime_data.isnull().sum()

state                     0
community              1177
communityname             0
fold                      0
population                0
                       ... 
LemasPctPolicOnPatr    1675
LemasGangUnitDeploy    1675
LemasPctOfficDrugUn       0
PolicBudgPerPop        1675
ViolentCrimesPerPop       0
Length: 127, dtype: int64

In [14]:
crime_data[crime_data.columns[crime_data.isnull().any()]]

,community,OtherPerCap,LemasSwornFT,LemasSwFTPerPop,LemasSwFTFieldOps,LemasSwFTFieldPerPop,LemasTotalReq,LemasTotReqPerPop,PolicReqPerOffic,PolicPerPop,...,PctPolicAsian,PctPolicMinor,OfficAssgnDrugUnits,NumKindsDrugsSeiz,PolicAveOTWorked,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,PolicBudgPerPop
0,NaN,0.36,0.03,0.13,0.96,0.17,0.06,0.18,0.44,0.13,...,0.1,0.07,0.02,0.57,0.29,0.06,0.04,0.9,0.5,0.14
1,NaN,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,81440,0.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6096,0.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,NaN,0.36,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990,NaN,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1991,80070,0.22,0.06,0.3,0.93,0.36,0.04,0.17,0.19,0.3,...,0,0.34,0.1,0.36,0.25,0.08,0.06,0.78,0,0.28
1992,72600,0.27,0.02,0.25,0.97,0.3,0.01,0.11,0.14,0.25,...,0,0.09,0.01,0.57,0.19,0.02,0.02,0.79,0,0.18


In [15]:
# fillNa values with median imputation
crime_data.fillna(crime_data.median(),inplace=True)
# #Convert community object into int64
# crime_data['community']=crime_data['community'].astype('int64')
crime_data

C:\Users\tanzi\AppData\Local\Temp\ipykernel_53912\3705086494.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  crime_data.fillna(crime_data.median(),inplace=True)


,state,community,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,LandArea,PopDens,PctUsePubTrans,PolicCars,PolicOperBudg,LemasPctPolicOnPatr,LemasGangUnitDeploy,LemasPctOfficDrugUn,PolicBudgPerPop,ViolentCrimesPerPop
0,8,48090.0,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,0.17,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,48090.0,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,0.07,...,0.02,0.12,0.45,0.08,0.03,0.75,0.5,0.00,0.15,0.67
2,24,48090.0,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,0.04,...,0.01,0.21,0.02,0.08,0.03,0.75,0.5,0.00,0.15,0.43
3,34,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,0.10,...,0.02,0.39,0.28,0.08,0.03,0.75,0.5,0.00,0.15,0.12
4,42,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,0.05,...,0.04,0.09,0.02,0.08,0.03,0.75,0.5,0.00,0.15,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12,48090.0,TempleTerracecity,10,0.01,0.40,0.10,0.87,0.12,0.16,...,0.01,0.28,0.05,0.08,0.03,0.75,0.5,0.00,0.15,0.09
1990,6,48090.0,Seasidecity,10,0.05,0.96,0.46,0.28,0.83,0.32,...,0.02,0.37,0.20,0.08,0.03,0.75,0.5,0.00,0.15,0.45
1991,9,80070,Waterburytown,10,0.16,0.37,0.25,0.69,0.04,0.25,...,0.08,0.32,0.18,0.08,0.06,0.78,0,0.91,0.28,0.23
1992,25,72600,Walthamcity,10,0.08,0.51,0.06,0.87,0.22,0.10,...,0.03,0.38,0.33,0.02,0.02,0.79,0,0.22,0.18,0.19


## Encode the dataset

In [ ]:
# Encode the object type in crime_data

#
X_data_ =


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split_test = StratifiedShuffleSplit(n_splits=1,test_size=0.2, random_state=23508893)
for train_index, test_index in split_test.split(X_data_enc, Y_data_enc):
    X_train_proj_full = X_data_enc[train_index,:]
    X_test_proj = X_data_enc[test_index,:]
    Y_train_proj_full = Y_data_enc[train_index]
    Y_test_proj = Y_data_enc[test_index]

split_validation = StratifiedShuffleSplit(n_splits=1,test_size=0.25, random_state=23508893)
for train_index, validation_index in split_validation.split(X_train_proj_full, Y_train_proj_full):
    X_train_proj = X_train_proj_full[train_index,:]
    X_validation_proj = X_train_proj_full[validation_index,:]
    Y_train_proj = Y_train_proj_full[train_index]
    Y_validation_proj = Y_train_proj_full[validation_index]

#reshape labels for sklearn
Y_train_proj_full =  Y_train_proj_full.reshape((Y_train_proj_full.shape[0],))
Y_train_proj = Y_train_proj.reshape((Y_train_proj.shape[0],))
Y_validation_proj= Y_validation_proj.reshape((Y_validation_proj.shape[0],))
Y_test_proj = Y_test_proj.reshape((Y_test_proj.shape[0],))


In [ ]:
print("Dataset\t\tPercent")
print("Training\t" + str(np.sum(crime_train_proj) / len(crime_train_proj)))
print("Validation\t" + str(np.sum(crime_validation_proj) / len(crime_validation_proj)))
print("Test\t\t" + str(np.sum(crime_test_proj) / len(crime_test_proj)))

In [ ]:
#peformance metric functions
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import numpy as np

#A list of keys for the dictionary returned by p1_metrics
metric_keys = ['mse','mae','r2']

def p2_metrics(y_true,y_pred,negation=False):
    if negation:
        sign = -1
    else:
        sign = 1
    return {
        'mse': sign*mean_squared_error(y_true,y_pred),
        'mae': sign*mean_absolute_error(y_true,y_pred),
        'r2': sign*r2_score(y_true,y_pred)}

#This wrapper can be used to return multiple performance metrics during cross-validation
def p2_metrics_scorer(clf,X,y_true):
    y_pred=clf.predict(X)
    return p2_metrics(y_true,y_pred,negation=True)

## 2.2 Linear Regression

1. Construct a linear model using Scikit-learn's `LinearRegression` method with default parameters.
2. Report the following performance metrics on the **training and validation sets**:
    *Mean Squared Error*, *Mean Absolute Error*, and the *Coefficient of Determination ($r^2$)*.
    
    You can use the function `p2_metrics` for this purpose. Is this model underfitting the data? Is so, why?
3. Report the weights (coefficients) of the linear model and their associated features in ascending order.

    Larger weights indicate that their corresponding features have more influence in the model. Moreover, negative weights correspond to variables having negative correlation with the response variable, and vice versa.
    
    Using this interpretation, describe the most significant features and their correlation.

## 2.3 Linear Regression and PCA

1. Perform principal component analysis on the **training data**. You may use Scikit-learn's `PCA` function for this, which **automatically centers** the data prior to PCA. Using PCA, *choose the number of components* for which the total explained variance is $\ge 99\%$, and report this.
2. After determining the correct number of components, apply the PCA transformation to the **validation** and **test** sets.
3. Create another model via `LinearRegression` but using the data transformed by PCA
Construct a linear model using Scikit-learn's `LinearRegression` method with default parameters.
4. Report the same performance metrics as in 2.2 on the **validation set**. How does the model's performance compare to that of the model in 2.2?

## 2.4 LASSO

Utilizing *cross-validation* you will Construct an $\ell_1$-regularized linear model using Scikit-learn's `LASSO`:
1. Using `GridSearchCV`, determine the best choice of the hyperparameter $\alpha$ out of values in the list `alphas` below.
2. Report the time required to perform cross-validation via `GridSearchCV`. Report the mean and standard deviation of the performance metrics for the best performing model along with its associated hyperparameter. You may use the function `collate_ht_results` for this purpose.
3. Report the weights (coefficients) of the LASSO model and their associated features in ascending order. Note that LASSO attempts to set as many weights to zero in order to create a more parsimonious model while still maintaining regression performance. How many weights are non-zero?
 
### Please Read!
There are a few parameters for the `GridSearchCV` and `RandomizedSearchCV` functions that should be set:
- `scoring` - This controls the strategy to evaluate the performance of the cross-validated model on the test set, set it to `p2_metrics_scorer`.
- `refit` - This will refit an estimator using the best found parameters on the whole dataset, set it to `"mse"`
- `cv` - This will enable you to reuse your CV splits created in Part 2.1
    `n_jobs` - Number of jobs to run in parallel, if you have more than one core on your device (you should), set this to as many as you'd like to use, or to `-1` if you want to use all available cores.
- `return_train_score` - Setting this to `False` will reduce computational time.

In [ ]:
alphas = 10**np.linspace(1,-3,50)*0.5

In [ ]:
#Summarizes model performance results produced during hyperparameter tuning
def collate_ht_results(ht_results,metric_keys=metric_keys,display=True):
    ht_stats=dict()
    for metric in metric_keys:
        ht_stats[metric+"_mean"] = ht_results.cv_results_["mean_test_"+metric][ht_results.best_index_]
        ht_stats[metric+"_std"] = metric_std = ht_results.cv_results_["std_test_"+metric][ht_results.best_index_]
        if display:
            print("test_"+metric,ht_stats[metric+"_mean"],"("+str(ht_stats[metric+"_std"])+")")
    return ht_stats

## 2.5 Multilayer Perceptron (MLP)

Utilizing *cross-validation* you will construct an MLP regression model using Scikit-learn's `MLPRegressor`:
1. Using `GridSearchCV`, determine the best choice of hyperparameters out of the following possible values:
- *Number of hidden layers*: [1, 2, 3]
- *Number of neurons per layer*: [10, 20, 50]
- *Learning rate*: [1e-5, 1e-4, 0.001, 0.01, 0.1, 0.5, 1, 5, 10, 50, 100]

2. Report the time required to perform cross-validation via `GridSearchCV`. Report the mean and standard deviation of the performance metrics for the best performing model along with its associated hyperparameter. You may use the function `collate_ht_results` for this purpose.

 
### Please Read!
In addition to utilizing the same `GridSearchCV` parameters as in 2.5, the `MLPRegressor` function should have the following parameters set:
- `max_iter` -  This controls the maximum number of rounds of backpropagation/gradient descent; set it to 10,000.
- `early_stopping` - This will reserve a portion of the training data tha can be used to evaluate convergence progress in order to stop training early; set it to `True`.

## 2.6 Final Models

1. Using the full training set (**training + validation**), train *two* linear regression models, one with and without PCA preprocessing, then apply them to the test set. For LASSO and MLP, you can utilize the best models found during cross-validation and just apply them to the test set.
2. Create a bar chart of the three regression metrics for each model on the same plot.
3. How do the models's performances compare? What do the metrics reveal about the dataset?